In [19]:
from Bio import Entrez
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
import json
import csv
import jsonlines
import tqdm 
import requests

# 设置你的邮箱地址（NCBI要求提供一个邮箱地址）
Entrez.email = "ynaxd@example.com"
API_KEY = '73664c0b675e8450f3cf90add93e69820808'
from ete3 import NCBITaxa
# ncbi_texa = NCBITaxa(taxdump_file="/Volumes/PortableSSD/data/bio/bio/taxdump_2015-12-01.tar.gz")
ncbi_texa = NCBITaxa(taxdump_file="/Volumes/PortableSSD/data/bio/bio/taxdump.tar.gz")



def get_lineage(taxid=57483):
    try:
        lineage = ncbi_texa.get_lineage(int(taxid))
        lineage = sorted(lineage)
        names = ncbi_texa.get_taxid_translator(lineage)
        lineages = [names[k] for k in lineage]
        organism = [v for v in lineages if v.endswith("virus")]
        viridae = [v for v in lineages if v.endswith("viridae")]
        organism = organism [0] if organism else None
        viridae = viridae[0] if viridae else None
        return organism, viridae
    except:
        return None, None

def get_protein_info(virus_name, protein_name):
    # 在 NCBI 中搜索特定病毒的蛋白信息
    search_query = f"{virus_name} and {protein_name} [Protein]"
    handle = Entrez.esearch(db="protein", term=search_query, retmax=1)
    record = Entrez.read(handle)
    handle.close()
    
    if record["Count"] == "0":
        return "", ""
    # 获取 GI 号
    gi_number = record["IdList"][0]
    # 根据 GI 号获取序列信息
    with Entrez.efetch(db="protein", id=gi_number, rettype="fasta", retmode="text") as handle:
        sequence_record = SeqIO.read(handle, "fasta")
    return gi_number, sequence_record

def get_taxid(term):
    url = f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=taxonomy&retmax=3&retmode=json&api_key={API_KEY}&term={term}"
    content = requests.get(url)
    body = json.loads(content.text)
    if "esearchresult" in body and "idlist" in body["esearchresult"]:
        return body["esearchresult"]["idlist"][0] if body["esearchresult"]["idlist"] else ""


Loading node names...
2533608 names loaded.
310365 synonyms loaded.
Loading nodes...
2533608 nodes loaded.
Linking nodes...
Tree is loaded.
Updating database: /Users/john/.etetoolkit/taxa.sqlite ...
 2533000 generating entries... 
Uploading to /Users/john/.etetoolkit/taxa.sqlite


Inserting synonyms:      25000 

Inserting synonyms:      310000 

Inserting taxids:       25000  

Inserting taxids:       2530000      

In [4]:
records = []
tlp = "/Volumes/PortableSSD/projects/EnvelopProtein/nc_zf/{}"
with open(tlp.format("env.csv")) as fp:
    with open(tlp.format("env_gi.csv"), "w") as wp:
        reader = csv.DictReader(fp)
        writer = csv.DictWriter(wp, fieldnames=("ID", "Name", "Family", "Pid","gi", "taxid", "family", "species"))
        writer.writeheader()
        for v in tqdm.tqdm(reader):
            gi, record = get_protein_info(v["Name"], v["ID"])
            v["taxid"]= get_taxid(v["Name"])
            v["gi"] = gi
            species, family = get_lineage(int(v["taxid"])) if v["taxid"] else ("", "")
            v["family"] = family
            v["species"] = species
            if record:
                records.append(record)
            writer.writerow(v)
SeqIO.write(records, tlp.format("env_gi.fasta"), "fasta")
            

93it [04:35,  2.96s/it]


1

In [47]:
get_lineage(11041)

('Rubivirus', 'Togaviridae')

In [55]:
records

['>sp|P17501.2|FUS_NPVAC RecName: Full=Major envelope glycoprotein; AltName: Full=gp64; Flags: Precursor\nMVSAIVLYVLLAAAAHSAFAAEHCNAQMKTGPYKIKNLDITPPKETLQKDVEITIVETDYNENVIIGYKG\nYYQAYAYNGGSLDPNTRVEETMKTLNVGKEDLLMWSIRQQCEVGEELIDRWGSDSDDCFRDNEGRGQWVK\nGKELVKRQNNNHFAHHTCNKSWRCGISTSKMYSRLECQDDTDECQVYILDAEGNPINVTVDTVLHRDGVS\nMILKQKSTFTTRQIKAACLLIKDDKNNPESVTREHCLIDNDIYDLSKNTWNCKFNRCIKRKVEHRVKKRP\nPTWRHNVRAKYTEGDTATKGDLMHIQEELMYENDLLKMNIELMHAHINKLNNMLHDLIVSVAKVDERLIG\nNLMNNSVSSTFLSDDTFLLMPCTNPPAHTSNCYNNSIYKEGRWVANTDSSQCIDFSNYKELAIDDDVEFW\nIPTIGNTTYHDSWKDASGWSFIAQQKSNLITTMENTKFGGVGTSLSDITSMAEGELAAKLTSFMFGHVVN\nFVIILIVILFLYCMIRNRNRQY\n\n']

In [14]:
def get_protein_info(virus_name, protein_name):
    # 在 NCBI 中搜索特定病毒的蛋白信息
    search_query = f"{virus_name} AND {protein_name} [Protein]"
    handle = Entrez.esearch(db="protein", term=search_query, retmax=1)
    record = Entrez.read(handle)
    handle.close()
    
    if record["Count"] == "0":
        return "", ""
    # 获取 GI 号
    gi_number = record["IdList"][0]
    # 根据 GI 号获取序列信息
    with Entrez.efetch(db="protein", id=gi_number, rettype="fasta", retmode="text") as handle:
        sequence_record = SeqIO.read(handle, "fasta")
    return gi_number, sequence_record

get_protein_info("SARS-CoV_S", "Human SARS coronavirus")

('', '')